In [ ]:
from utils import *
import glob
from IPython.display import Markdown

In [ ]:
pdf_paths = glob.glob('./data/*')
financial_reports_vector_store = create_vector_store(pdf_paths)
agent_manager = create_agentic_team(financial_reports_vector_store, web_search_required = False)

In [ ]:
q = "What was NVIDIA's gross profit in January 2025 ? How much percentage has it increased from last year ?"
ans = agent_manager.run(q)

In [ ]:
Markdown(str(ans))

In [ ]:
q = 'How much free cash flow did NVIDIA generate in Q4 FY25?'
ans = agent_manager.run(q)

In [ ]:
Markdown(str(ans))

In [ ]:
agent_manager.memory.steps

In [ ]:
import os
from smolagents import LiteLLMModel
from dotenv import load_dotenv
load_dotenv()
model = LiteLLMModel(model_id="groq/llama3-70b", api_key=os.environ["GROQ_API_KEY"])


In [ ]:
model.invoke('hello')

In [ ]:
model.chat('hello')

In [ ]:
# Prepare input
messages = [{"role": "user", "content": "Explain the impact of AI on renewable energy."}]

# Invoke the model
response = model(messages)

# Print the response
print("Model Response:", response)


In [ ]:
import glob
from IPython.display import Markdown
from docling.document_converter import DocumentConverter
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document 
from dotenv import load_dotenv
load_dotenv()

In [ ]:
pdf_paths = glob.glob('./data/*')
pdf_paths

In [ ]:
def create_vector_store(pdf_paths):
    """
    Create a FAISS vector store from multiple PDFs.

    Args:
        pdf_paths (list): List of PDF file paths.

    Returns:
        FAISS vector store
    """

    # PDF Exraction
    pdf_extracted_text = []
    converter = DocumentConverter()
    for pdf in pdf_paths:
        result = converter.convert(pdf)
        markdown_text = result.document.export_to_markdown()
        pdf_extracted_text+=[markdown_text]

    # Convert to documents
    split_documents = [Document(page_content=chunk) for chunk in pdf_extracted_text]

    # Generate embeddings
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

    # Create FAISS vector store
    vectorstore = FAISS.from_documents(split_documents, embeddings)

    return vectorstore

In [ ]:
len(pdf_extracted_text)

In [ ]:
pdf_extracted_text

In [ ]:
# Split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=3000, chunk_overlap=30, separator="\n")
split_text_chunks = text_splitter.split_text(markdown_text)
split_documents = [Document(page_content=chunk) for chunk in split_text_chunks]

# Generate embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Create FAISS vector store
vectorstore = FAISS.from_documents(split_documents, embeddings)

In [ ]:
from utils import *

In [ ]:
manager1 = create_agentic_team(vectorstore, web_search_required = False)

In [ ]:
pdf_paths = glob.glob('./data/*')
vs1 = create_vector_store(pdf_paths)

In [ ]:
# ans = manager1.run("What was YoY revenue growth rate for Apple in quarter 4 ?")
ans = manager1.run("By how much have Apples Net income increased in December 2024 compared to last year ?")

In [ ]:
Markdown(ans)

In [ ]:
# Vector Store

In [ ]:
all_documents = []
# Load and extract text from all PDFs
for pdf_path in pdf_paths:
    loader = PyPDFLoader(file_path=pdf_path)
    documents = loader.load()
    all_documents.extend(documents)

# Split text into chunks
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator="\n")
split_documents = text_splitter.split_documents(all_documents)

# Generate embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Create FAISS vector store
vectorstore = FAISS.from_documents(split_documents, embeddings)

return vectorstore